# Pasiruošimas

In [ ]:
!git clone https://github.com/niu-lab/gclust

In [ ]:
%cd gclust
!make

In [ ]:
# Step 1: Download seqkit
!wget https://github.com/shenwei356/seqkit/releases/download/v2.4.0/seqkit_linux_amd64.tar.gz

# Step 2: Extract the binary from the tar.gz file
!tar -xzvf seqkit_linux_amd64.tar.gz

# Step 3: Make seqkit executable
!chmod +x seqkit

In [ ]:
!sudo dpkg --configure -a

# Step 1: Install FastTree
!apt-get install -y fasttree

In [ ]:
!sudo apt update
!sudo apt install mafft

In [6]:
!pip install ete3
from ete3 import Tree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 38.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ete3: filename=ete3-3.1.3-py3-none-any.whl size=2273787 sha256=da2f9a60d8f9fcaf5eb44032b123709626dba69889d9a011d67a54e77deccd44
  Stored in directory: /root/.cache/pip/wheels/a0/72/00/1982bd848e52b03079dbf800900120bc1c20e92e9a1216e525
Successfully built ete3


In [7]:
import os

# Define the path for the outputs folder
outputs_path = "/content/gclust/outputs"

# Create the folder if it doesn't exist
os.makedirs(outputs_path, exist_ok=True)

print(f"Folder created (if it didn't exist): {outputs_path}")

Folder created (if it didn't exist): /content/gclust/outputs


# Apdorojimas

In [8]:
#Sort
!perl script/sortgenome.pl --genomes-file /content/gclust/outputs/all_sequences.fasta --sortedgenomes-file /content/gclust/outputs/sorted_all.fasta

#Cluster
!./gclust -minlen 20 -both -nuc -threads 8 -ext 1 -sparse 2 -memiden 97 /content/gclust/outputs/sorted_all.fasta > /content/gclust/outputs/clustering.out

#Get ids of the representatives
!grep ".*\*" /content/gclust/outputs/clustering.out | cut -d'>' -f2 | awk '{print $1}' | sed 's/\.\.\.$//' > /content/gclust/outputs/representative_ids.txt

#Use seqkit to extract sequences
!./seqkit grep -i -f /content/gclust/outputs/representative_ids.txt /content/gclust/outputs/all_sequences.fasta -o /content/gclust/outputs/result.fasta

total genomes:	383
sorting....
genomes sorting by size finished
writing output ...

Gclust version 1.0


Genomes: 383
Maximum Length: 30448
Minimum Length: 27112
Average Length: 29677



Load genomes: 383

chunk: 383

====
S 11366874
startpos 383
descr 383
====

Creating suffix array ......

suffix sorting ....
suffix sorting done ....
N=11366876
logN 	23
N/K=5683438
M.size()=3855468
NKm1= 5683437

Finished creating suffix array ......

...... 0 done
...... 100 done
...... 200 done
...... 300 done

Output clustering information ......

Total clusters: 114
The finish.

time:294 s
[INFO] 114 patterns loaded from file


In [9]:
#Translate
!./seqkit translate /content/gclust/outputs/aligned.fasta -o /content/gclust/outputs/translated_proteins.fasta

#Filter shorter than 800
!./seqkit seq -g -m 800 /content/gclust/outputs/translated_proteins.fasta -o /content/gclust/outputs/filtered_proteins.fasta

#mafft alignment
!mafft --maxiterate 1000 --localpair /content/gclust/outputs/filtered_proteins.fasta > /content/gclust/outputs/output_aligned.fasta

# File paths
input_file = "/content/gclust/outputs/output_aligned.fasta"
output_file = "/content/gclust/outputs/output_cleaned_aligned.fasta"

# Open the input file, clean it, and write to the output file
with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        # Remove spaces and ':' characters
        cleaned_line = line.replace(" ", "_").replace(":", "_")
        outfile.write(cleaned_line)

outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
rescale = 1
All-to-all alignment.
tbfast-pair (aa) Version 7.490
alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  140 / 150
done.

Progressive alignment ... 
STEP   144 /149 
Reallocating..done. *alloclen = 4907
STEP   149 /149 
done.
tbfast (aa) Version 7.490
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
rescale = 1

  140 / 150
Segment   1/  1    1-2048
STEP 003-034-1  identical.   
Converged.

done
dvtditr (aa) Version 7.490
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


St

In [12]:
#Generate Tree
!FastTree -gamma /content/gclust/outputs/output_cleaned_aligned.fasta > /content/gclust/outputs/phylogenetic_tree.txt

FastTree Version 2.1.11 Double precision (No SSE3)
Alignment: /content/gclust/outputs/output_cleaned_aligned.fasta
Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories
Ignored unknown character X (seen 4 times)
Initial topology in 0.51 seconds
Refining topology: 29 rounds ME-NNIs, 2 rounds ME-SPRs, 14 rounds ML-NNIs
Total branch-length 3.610 after 4.96 sec
ML-NNI round 1: LogLk = -47228.830 NNIs 25 max delta 15.44 Time 18.28
Switched to using 20 rate categories (CAT approximation)
Rate categories were divided by 0.877 so that average rate = 1.0
CAT-based log-likelihoods may not be comparable across runs
ML-NNI round 2: LogLk = -43156.423 NNIs 9 max delta 7.02 Time 30.89
ML-NNI round 3: LogLk = -43149.948 NNIs 1 max delta 0.70 Time 33.71
ML-NNI round 4: LogLk = -43149.936 NNIs 0 max delta 

In [15]:
tree = Tree("/content/gclust/outputs/phylogenetic_tree.txt")

# Print all leaf names
print("Leaf names in the tree:")
print(tree.get_leaf_names())

# Load the tree from the Newick file
tree = Tree("/content/gclust/outputs/phylogenetic_tree.txt")

# Specify the outgroup (camel virus identifier)
camel_virus_label = "lcl|Query_4392169_4901-8458_MN514967.1_Dromedary_camel_coronavirus_HKU23_isolate_DcCoV-HKU23/camel/Nigeria/NV1385/2016"

# Set the camel virus as the root of the tree
tree.set_outgroup(camel_virus_label)

# Save the rooted tree back to a file
tree.write(outfile="/content/gclust/outputs/rooted_phylogenetic_tree.txt")

Leaf names in the tree:
['lcl|Query_4392097_2719-8484_KY417149.1_Bat_SARS-like_coronavirus_isolate_Rs4255', 'lcl|Query_4392066_2719-8484_KY417152.1_Bat_SARS-like_coronavirus_isolate_Rs9401', 'lcl|Query_4392062_2719-8484_KY417147.1_Bat_SARS-like_coronavirus_isolate_Rs4237', 'lcl|Query_4392099_2809-8574_OQ503500.1_Severe_acute_respiratory_syndrome-related_coronavirus_isolate_4105', 'lcl|Query_4392074_2719-8484_KY417150.1_Bat_SARS-like_coronavirus_isolate_Rs4874', 'lcl|Query_4392076_2719-8484_KY417143.1_Bat_SARS-like_coronavirus_isolate_Rs4081', 'lcl|Query_4392064_2719-8484_KY417148.1_Bat_SARS-like_coronavirus_isolate_Rs4247', 'lcl|Query_4392070_2719-8484_KY417151.1_Bat_SARS-like_coronavirus_isolate_Rs7327', 'lcl|Query_4392061_2719-8484_KY417145.1_Bat_SARS-like_coronavirus_isolate_Rf4092', 'lcl|Query_4392049_2511-8273_OK017858.1_Sarbecovirus_sp._isolate_YN2020H', 'lcl|Query_4392116_2696-8461_OQ503501.1_Severe_acute_respiratory_syndrome-related_coronavirus_isolate_4230', 'lcl|Query_4392131

# Testavimui

In [ ]:
# Generate the Phylogenetic Tree
!FastTree -gamma /content/gclust/outputs/output_cleaned_aligned.fasta > /content/gclust/outputs/phylogenetic_tree.txt

In [ ]:
#Sort
!perl script/sortgenome.pl --genomes-file /content/gclust/outputs/all_sequences.fasta --sortedgenomes-file /content/gclust/outputs/sorted_all.fasta

total genomes:	454
sorting....
genomes sorting by size finished
writing output ...


In [ ]:
#Cluster
!./gclust -minlen 20 -both -nuc -threads 8 -ext 1 -sparse 2 -memiden 97 /content/gclust/outputs/sorted_all.fasta > /content/gclust/outputs/clustering.out


Gclust version 1.0


Genomes: 454
Maximum Length: 30799
Minimum Length: 27141
Average Length: 29660



Load genomes: 454

chunk: 454

====
S 13466347
startpos 454
descr 454
====

Creating suffix array ......

suffix sorting ....
suffix sorting done ....
N=13466350
logN 	23
N/K=6733175
M.size()=4273012
NKm1= 6733174

Finished creating suffix array ......

...... 0 done
...... 100 done
...... 200 done
...... 300 done
...... 400 done

Output clustering information ......

Total clusters: 148
The finish.

time:354 s


In [ ]:
#get ids of the representatives
!grep ".*\*" /content/gclust/outputs/clustering.out | cut -d'>' -f2 | awk '{print $1}' | sed 's/\.\.\.$//' > /content/gclust/outputs/representative_ids.txt

In [ ]:
#Use seqkit to extract sequences
!./seqkit grep -i -f /content/gclust/outputs/representative_ids.txt /content/gclust/outputs/all_sequences.fasta -o /content/gclust/outputs/result.fasta

[INFO] 148 patterns loaded from file


In [ ]:
#Translate
!./seqkit translate /content/gclust/outputs/aligned.fasta -o /content/gclust/outputs/translated_proteins.fasta

In [ ]:
#Filter shorter than 800
!./seqkit seq -g -m 800 /content/gclust/outputs/translated_proteins.fasta -o /content/gclust/outputs/filtered_proteins.fasta

In [ ]:
!mafft --maxiterate 1000 --localpair /content/gclust/outputs/filtered_proteins.fasta > /content/gclust/outputs/output_aligned.fasta

outputhat23=16
treein = 0
compacttree = 0
stacksize: -1 kb
rescale = 1
All-to-all alignment.
tbfast-pair (aa) Version 7.490
alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  140 / 150
done.

Progressive alignment ... 
STEP   144 /149 
Reallocating..done. *alloclen = 4907
STEP   149 /149 
done.
tbfast (aa) Version 7.490
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
rescale = 1

  140 / 150
Segment   1/  1    1-2048
STEP 003-034-1  identical.   
Converged.

done
dvtditr (aa) Version 7.490
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Stra

In [ ]:
# File paths
input_file = "/content/gclust/outputs/output_aligned.fasta"
output_file = "/content/gclust/outputs/output_cleaned_aligned.fasta"

# Open the input file, clean it, and write to the output file
with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        # Remove spaces and ':' characters
        cleaned_line = line.replace(" ", "_").replace(":", "_")
        outfile.write(cleaned_line)

In [ ]:
# Step 2: Generate the Phylogenetic Tree
# Assuming you already have an aligned file (e.g., `cleaned_aligned_proteins.fasta`)
!FastTree -gamma /content/gclust/outputs/output_cleaned_aligned.fasta > /content/gclust/outputs/phylogenetic_tree.txt

In [ ]:
!pip install ete3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 45.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ete3: filename=ete3-3.1.3-py3-none-any.whl size=2273787 sha256=57fa5f21bf18e49a0957b62feae3558c27c3c858f793eb4bb617018c11a50e94
  Stored in directory: /root/.cache/pip/wheels/a0/72/00/1982bd848e52b03079dbf800900120bc1c20e92e9a1216e525
Successfully built ete3


In [ ]:
from ete3 import Tree

In [ ]:
from ete3 import Tree

# Load the tree file
tree = Tree("/content/gclust/outputs/phylogenetic_tree.txt")

# Print all leaf names
print("Leaf names in the tree:")
print(tree.get_leaf_names())


Leaf names in the tree:
['lcl|Query_4358112_2719-8484_KY417149.1_Bat_SARS-like_coronavirus_isolate_Rs4255', 'lcl|Query_4358082_2719-8484_KY417152.1_Bat_SARS-like_coronavirus_isolate_Rs9401', 'lcl|Query_4358077_2719-8484_KY417147.1_Bat_SARS-like_coronavirus_isolate_Rs4237', 'lcl|Query_4358114_2809-8574_OQ503500.1_Severe_acute_respiratory_syndrome-related_coronavirus_isolate_4105', 'lcl|Query_4358089_2719-8484_KY417150.1_Bat_SARS-like_coronavirus_isolate_Rs4874', 'lcl|Query_4358091_2719-8484_KY417143.1_Bat_SARS-like_coronavirus_isolate_Rs4081', 'lcl|Query_4358079_2719-8484_KY417148.1_Bat_SARS-like_coronavirus_isolate_Rs4247', 'lcl|Query_4358085_2719-8484_KY417151.1_Bat_SARS-like_coronavirus_isolate_Rs7327', 'lcl|Query_4358076_2719-8484_KY417145.1_Bat_SARS-like_coronavirus_isolate_Rf4092', 'lcl|Query_4358064_2511-8273_OK017858.1_Sarbecovirus_sp._isolate_YN2020H', 'lcl|Query_4358131_2696-8461_OQ503501.1_Severe_acute_respiratory_syndrome-related_coronavirus_isolate_4230', 'lcl|Query_4358146

In [ ]:
# Load the tree from the Newick file
tree = Tree("/content/gclust/outputs/phylogenetic_tree.txt")

# Specify the outgroup (camel virus identifier)
camel_virus_label = "lcl|Query_4358037_4901-8458_MN514967.1_Dromedary_camel_coronavirus_HKU23_isolate_DcCoV-HKU23/camel/Nigeria/NV1385/2016"

# Set the camel virus as the root of the tree
tree.set_outgroup(camel_virus_label)

# Save the rooted tree back to a file
tree.write(outfile="/content/gclust/outputs/rooted_phylogenetic_tree.txt")


In [16]:
import shutil
shutil.make_archive('/content/outputs', 'zip', '/content/gclust/outputs')

'/content/outputs.zip'